# IMPORTING LIBRARIES AND CONNECTING MYSQL

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector

db = mysql.connector.connect(host = "localhost",
                             username = "root",
                             password = "**********",
                             database = "pizzasales",
                             consume_results=True
                            )
cur = db.cursor()
cnx = mysql.connector.connect()

# BASIC QUERIES 

## Retrieve the total number of orders placed

In [5]:
query = """ select count(order_id) from orders """
cur.execute(query)
data = cur.fetchall()
pd.DataFrame(data,columns=["Total_Orders"])

,Total_Orders
0,42700


## Calculate the total revenue generated from Pizza Sales

In [7]:
query = """ select round(sum(order_details.quantity * pizzas.price),2)
from order_details join pizzas
on order_details.pizza_id = pizzas.pizza_id """
cur.execute(query)
data = cur.fetchall()
pd.DataFrame(data, columns=["Total Revenue"])

,Total Revenue
0,1635720.1


## Identify the highest priced pizza

In [9]:
query = """ select pizza_types.name,pizzas.price
from pizza_types join pizzas
on pizza_types.pizza_type_id = pizzas.pizza_type_id
order by pizzas.price desc limit 1 """
cur.execute(query)
data = cur.fetchall()
data

[('The Greek Pizza', 35.95)]

## Determine the distribution of orders by hour of the day

In [11]:
query = """ select hour(time) , count(order_id)
from orders group by hour(time)
order by count(order_id) desc """
cur.execute(query)
data = cur.fetchall()
pd.DataFrame(data,columns=["Hour_of_day","Order_count"])

,Hour_of_day,Order_count
0,12,5040
1,13,4910
2,18,4798
3,17,4672
4,19,4018
5,16,3840
6,20,3284
7,14,2944
8,15,2936
9,11,2462


## Find the category wise distribution of pizzas

In [13]:
query = """ select category, count(name) from Pizza_types
group by category"""
cur.execute(query)
data = cur.fetchall()
pd.DataFrame(data,columns=["Category","Pizza_Type_Count"])

,Category,Pizza_Type_Count
0,Chicken,6
1,Classic,8
2,Supreme,9
3,Veggie,9


# INTERMEDIATE QUERIES

## Identify the most common pizza size ordered

In [16]:
query = """ select pizzas.size,
count(order_details.order_id) as order_count
from pizzas join order_details
on pizzas.pizza_id=order_details.pizza_id
group by pizzas.size order by order_count desc """
cur.execute(query)
data = cur.fetchall()
pd.DataFrame(data, columns=["Pizza_Size","Order_Count"],index=[1,2,3,4,5])

,Pizza_Size,Order_Count
1,L,37052
2,M,30770
3,S,28274
4,XL,1088
5,XXL,56


## List the top 5 most ordered pizza types along with their quantities

In [18]:
## Method - 1 - Direct by Pizza Type id
query = """ select pizza_id,sum(quantity) as quantity
from order_details 
group by pizza_id
order by quantity desc limit 5 """
cur.execute(query)
data = cur.fetchall()
pd.DataFrame(data, columns = ["Pizza_Type_size","Order_Quantity"],index=[1,2,3,4,5])

,Pizza_Type_size,Order_Quantity
1,big_meat_s,3828
2,thai_ckn_l,2820
3,five_cheese_l,2818
4,four_cheese_l,2632
5,classic_dlx_m,2362


In [19]:
## Method -2 - Showing pizza Names
query = """ select pizza_types.name,
sum(order_details.quantity) as quantity
from pizza_types join pizzas
on pizza_types.pizza_type_id = pizzas.pizza_type_id
join order_details 
on pizzas.pizza_id = order_details.pizza_id
group by pizza_types.name
order by quantity desc limit 5 """
cur.execute(query)
data = cur.fetchall()
pd.DataFrame(data,columns=["Pizza_Name","Order_quantity"],index = [1,2,3,4,5])

,Pizza_Name,Order_quantity
1,The Classic Deluxe Pizza,4906
2,The Barbecue Chicken Pizza,4864
3,The Hawaiian Pizza,4844
4,The Pepperoni Pizza,4836
5,The Thai Chicken Pizza,4742


## Join the necessary tables to find the total quantity of each pizza category ordered

In [21]:
query = """ select pizza_types.category, 
sum(order_details.quantity) as quantity
from pizza_types join pizzas
on pizza_types.pizza_type_id = pizzas.pizza_type_id
join order_details on 
 pizzas.pizza_id = order_details.pizza_id 
 group by pizza_types.category
 order by quantity desc"""
cur.execute(query)
data = cur.fetchall()
pd.DataFrame(data, columns=["Pizza_category","Quantity"],index=[1,2,3,4])

,Pizza_category,Quantity
1,Classic,29776
2,Supreme,23974
3,Veggie,23298
4,Chicken,22100


### Group the orders by date and calculate the average number of Pizzas ordered per day

In [23]:
query = """ select round(avg(q)) from
(select orders.date, sum(order_details.quantity) as q
from orders join order_details 
on orders.order_id=order_details.order_id
group by orders.date) as oq """
cur.execute(query)
data = cur.fetchall()
print("Avg. no. of Pizzas ordered per day:",data)

Avg. no. of Pizzas ordered per day: [(Decimal('554'),)]


### Determine the top 3 most ordered pizza types based on revenue

In [25]:
query = """ select pizza_types.name as pn, 
round(sum(order_details.quantity * pizzas.price)) as rev
from pizza_types join pizzas
on pizza_types.pizza_type_id = pizzas.pizza_type_id
join order_details 
on pizzas.pizza_id=order_details.pizza_id
group by pn
order by rev desc limit 3"""
cur.execute(query)
data = cur.fetchall()
pd.DataFrame(data,columns=["pizza_type","revenue"],index=[1,2,3])

,pizza_type,revenue
1,The Thai Chicken Pizza,86868.0
2,The Barbecue Chicken Pizza,85536.0
3,The California Chicken Pizza,82819.0


# ADVANCED QUERIES

### Calculate the percentage contribution of each pizza type to total revenue

In [28]:
query = """ select pizza_types.category as pc,
round(sum(order_details.quantity*pizzas.price)/
(select sum(order_details.quantity*pizzas.price) 
from order_details join pizzas
on order_details.pizza_id=pizzas.pizza_id)*100,2) as rev
from pizza_types join pizzas
on pizza_types.pizza_type_id=pizzas.pizza_type_id
join order_details
on pizzas.pizza_id = order_details.pizza_id
group by pc order by rev desc """
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns=["pizza_type","percentage_contri"])
df

,pizza_type,percentage_contri
0,Classic,26.91
1,Supreme,25.46
2,Chicken,23.96
3,Veggie,23.68


### Analyze cumulative revenue generated over time

In [48]:
query = """ select date,sum(rev) over(order by date) as cum_val
from (select orders.date, 
round(sum(order_details.quantity*pizzas.price)) as rev
from orders join order_details
on orders.order_id=order_details.order_id
join pizzas on order_details.pizza_id=pizzas.pizza_id
group by orders.date) as sales """
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns=["Order_date","Cumulative_revenue"])
df.head()

,Order_date,Cumulative_revenue
0,2015-01-01,10855.0
1,2015-01-02,21783.0
2,2015-01-03,32433.0
3,2015-01-04,39455.0
4,2015-01-05,47719.0


### Determine top 3 most ordered pizza type based on revenue for each pizza category

In [52]:
query = """ select category,name,rev from
(select category,name,rev,
rank() over(partition by category order by rev) as rn from
(select pizza_types.category,pizza_types.name,
round(sum(order_details.quantity*pizzas.price)) as rev
from pizza_types join pizzas
on pizza_types.pizza_type_id=pizzas.pizza_type_id
join order_details
on pizzas.pizza_id=order_details.pizza_id
group by pizza_types.category,pizza_types.name
order by pizza_types.category,pizza_types.name) as sales) as st
where rn<=3 """
cur.execute(query)
data = cur.fetchall()
pd.DataFrame(data,columns=["category","name","revenue"])

,category,name,revenue
0,Chicken,The Chicken Pesto Pizza,33404.0
1,Chicken,The Chicken Alfredo Pizza,33800.0
2,Chicken,The Southwest Chicken Pizza,69412.0
3,Classic,"The Pepperoni, Mushroom, and Peppers Pizza",37669.0
4,Classic,The Big Meat Pizza,45936.0
5,Classic,The Napolitana Pizza,48174.0
6,Supreme,The Brie Carre Pizza,23177.0
7,Supreme,The Spinach Supreme Pizza,30556.0
8,Supreme,The Calabrese Pizza,31868.0
9,Veggie,The Green Garden Pizza,27912.0
